## ひろしまQuest2020#stayhome【球種予測部門】
#### all_merge_22A
- 22相当に戻す

#### all_merge_23
- 2017年の集計がない選手はNaNに

#### all_merge_22
- 2017年の結果を1/4ずつ集計して予測

#### all_merge_21
- 19相当に戻した

#### all_merge_20
- player同士の組み合わせを削除

#### all_merge_19
- コースの種類のみ

#### all_merge_18
- コースの種類、1球前のボール、ファウル数

#### all_merge_17
- イニング時間、打席時間、平均投球間隔(差)、サヨナラ、延長戦

#### all_merge_16
- 球団ごとの年棒ランク
- 選手IDごとの2017年のコース実績

#### all_merge_15
- 左右で球種/コースを別に集計

#### all_merge_14
- 左右で球種を別に集計

#### all_merge_13
- 前の投球からの時間差

In [232]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 300)

### 管理番号

In [233]:
model_No = '22A'

### sub-modelを使用するかどうか

In [234]:
use_sub_model = True

### n/4

In [235]:
sample_No = 4

In [265]:
ALL_MERGE = 'intermediate/{}/all_merge_{}_{}.f'.format(model_No, model_No, sample_No)
SUBMIT = 'submit/{}/ball_{}.csv'.format(model_No, model_No)
SUBMIT_F = 'submit/{}/ball_{}_{}.f'.format(model_No, model_No, sample_No)

if use_sub_model:
    ALL_MERGE = 'intermediate/{}/all_merge_{}_{}_sub.f'.format(model_No, model_No, sample_No)
    SUBMIT = 'submit/{}/ball_{}_sub.csv'.format(model_No, model_No)

OUT_SUBMODEL = 'intermediate/{}/ball_predict_{}_{}.f'.format(model_No, model_No, sample_No)

In [237]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 211)

In [238]:
if use_sub_model:
    all_pitch['predict_high_str'] = all_pitch['predict_0'] + all_pitch['predict_3'] + all_pitch['predict_6'] 
    all_pitch['predict_high_ball'] = all_pitch['predict_9'] + all_pitch['predict_10'] 
    all_pitch['predict_mid_str'] = all_pitch['predict_1'] + all_pitch['predict_4'] + all_pitch['predict_7'] 
    all_pitch['predict_low_str'] = all_pitch['predict_2'] + all_pitch['predict_5'] + all_pitch['predict_8'] 
    all_pitch['predict_low_ball'] = all_pitch['predict_11'] + all_pitch['predict_12'] 

    all_pitch['predict_left_str'] = all_pitch['predict_0'] + all_pitch['predict_1'] + all_pitch['predict_2'] 
    all_pitch['predict_left_ball'] = all_pitch['predict_9'] + all_pitch['predict_11'] 
    all_pitch['predict_center_str'] = all_pitch['predict_3'] + all_pitch['predict_4'] + all_pitch['predict_5'] 
    all_pitch['predict_right_str'] = all_pitch['predict_6'] + all_pitch['predict_7'] + all_pitch['predict_8'] 
    all_pitch['predict_right_ball'] = all_pitch['predict_10'] + all_pitch['predict_12'] 

In [239]:
if use_sub_model:
    all_pitch.drop(columns=[
        'predict_straight', 'predict_curve', 'predict_slider', 'predict_shoot',
        'predict_fork', 'predict_changeup', 'predict_sinker', 'predict_cutball',
        'predict_0','predict_1','predict_2','predict_3','predict_4','predict_5','predict_6',
        'predict_7','predict_8','predict_9','predict_10','predict_11','predict_12'
    ], inplace=True)

In [240]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 200)


,No,ball,course,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_high_str,bc_high_ball,bc_mid_str,bc_low_str,bc_low_ball,bc_left_str,bc_left_ball,bc_center_str,bc_right_str,bc_right_ball,first,second,third,base_cnt,top_bot,role,pos_pit,date_from_opening,elapsed_min,min_diff,total_inning,elapsed_from_inning,ballnum_from_inning,elapsed_batter,ave_elapsed_game,ave_elapsed_inning,ave_elapsed_batter,diff_elapsed_batter,diff_elapsed_inning,diff_elapsed_game,sayonara,extention,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_count_sum,pre_ball_foul,pre_ball_ball,pre_ball_strike,pre_foul_sum,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,firm_pit,height_pit,weight_pit,draft_order_pit,salary_pit,foreigner_pit,company_pit,univ_pit,highsch_pit,age_pit,play_year_pit,salary_rank_pit,rank_year_pit,rank_x_year_pit,salary_year_pit,salary_x_year_pit,bmi_pit,curve,slider,shoot,fork,changeup,sinker,cutball,total,pit_game_cnt,pit_inning_cnt,pit_batter_cnt,high_str,high_ball,mid_str,low_str,low_ball,left_str,left_ball,center_str,right_str,right_ball,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_rank_bat,rank_year_bat,rank_x_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,rank_year_dif_p-b,rank_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,rank_year_dif_p-c,rank_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,rank_year_dif_b-c,rank_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R,predict_high_str,predict_high_ball,predict_mid_str,predict_low_str,predict_low_ball,predict_left_str,predict_left_ball,predict_center_str,predict_right_str,predict_right_ball
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.237468,0.325101,0.209547,0.145227,0.092978,0.027829,0.113620,0.135883,0.221489,0.204959,0.174254,0.263415,0.242045,0.294634,0.157681,0.115370,0.190270,0,0,0,0,0,1,0,0,0.000000,NaN,0,0.000000,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,196,100,NaN,24000,1,0,0,0,29,6.0,5.5,0.916667,33.0,4000.0,144000.0,26.03082,0.368821,0.292776,0.114068,0.083650,0.04943,0.0,0.0,502.0,8.0,57.0,120.0,0.159363,0.201195,0.219124,0.191235,0.229084,0.183267,0.239044,0.205179,0.181275,0.191235,0,176,74,5.0,15000,0,1,1,0,32,8.0,2.5,0.312500,20.0,1875.0,120000.0,23.889463,66.0,16.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0,0.193606,0.224174,0.244037,0.171968,0.166215,0.246949,0.217751,0.207264,0.155398,0.172638
1,1,0.0,8.0,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.210923,0.247198,0.177502,0.320359,0.147952,0.039943,0.116772,0.106283,0.230235,0.157397,0.138195,0.367890,0.184621,0.355720,0.111331,0.105923,0.242405,0,0,0,0,0,1,0,0,0.200000,0.200000,0,0.200000,2,0.200000,0.200000,0.200000,0.200000,0.000000,0.000000,0.000000,0,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0.0,0.0,1.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,196,100,NaN,24000,1,0,0,0,29,6.0,5.5,0.916667,33.0,4000.0,144000.0,26.03082,0.368821,0.292776,0.114068,0.083650,0.04943,0.0,0.0,502.0,8.0,57.0

### train
- 行数: 257117

In [241]:
train = all_pitch.dropna(subset=['course'])
train.shape

(257117, 200)

#### 特徴量に加えた部分を抜く
- train_pitch_1 = train_pitch.query('index <= 60000')
- train_pitch_2 = train_pitch.query('index > 60000 & index <= 120000')
- train_pitch_3 = train_pitch.query('index > 120000 & index <= 180000')
- train_pitch_4 = train_pitch.query('index > 180000')

In [242]:
if sample_No == 1:
    train = train.query('index > 60000')
elif sample_No == 2:
    train = train.query('index <= 60000 | index > 120000')
elif sample_No == 3:
    train = train.query('index <= 120000 | index > 180000')
elif sample_No == 4:
    train = train.query('index <= 180000')
train.shape

(180001, 200)

### test
- 行数: 521650

In [243]:
test = all_pitch[all_pitch['course'].isnull()]
test.shape

(521650, 200)

In [244]:
del all_pitch
gc.collect()

49

In [245]:
train_d = train.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

In [246]:
test_d = test.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

### LGBM学習
- multiclass : クラス数=8

In [247]:
lgb_param = {
    'objective' : 'multiclass',
    'boosting_type': 'gbdt',
    'metric' : 'multi_logloss',
    'num_class' : 8,
    'seed' : 0,
    'learning_rate' : 0.1,
    'lambda_l1': 6.9923570049658075, 
    'lambda_l2': 0.002378623984798833, 
    'num_leaves': 18, 
    'feature_fraction': 0.45199999999999996, 
    'bagging_fraction': 0.9799724836460725, 
    'bagging_freq': 4, 
    'min_child_samples': 20
}

In [248]:
def lightgbm_cv(lgb_param, lgb_train):
    cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

    num_boost_round = len(cv_results['multi_logloss-mean'])
    print('Best num_boost_round:', num_boost_round)
    best_cv_score = cv_results['multi_logloss-mean'][-1]
    print('Best CV score:', best_cv_score)
    best_iter = int(num_boost_round * 1.1)
    return best_iter

### Feature Importance

In [249]:
def feature_importance(lgb_model):
    fi = lgb_model.feature_importance()
    fn = lgb_model.feature_name()
    df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
    df_feature_importance.sort_values('feat_imp', inplace=True)
    return df_feature_importance

In [250]:
%%time
lgb_train = lgb.Dataset(train_d, train['ball'])
best_iter = lightgbm_cv(lgb_param, lgb_train)

[100]	cv_agg's multi_logloss: 1.16235 + 0.00210774
[200]	cv_agg's multi_logloss: 1.13759 + 0.00206984
[300]	cv_agg's multi_logloss: 1.12941 + 0.001952
[400]	cv_agg's multi_logloss: 1.12488 + 0.00167185
[500]	cv_agg's multi_logloss: 1.12208 + 0.00165066
[600]	cv_agg's multi_logloss: 1.1201 + 0.0014466
[700]	cv_agg's multi_logloss: 1.11897 + 0.00134307
[800]	cv_agg's multi_logloss: 1.11816 + 0.00129738
[900]	cv_agg's multi_logloss: 1.11777 + 0.00126169
[1000]	cv_agg's multi_logloss: 1.11767 + 0.00125578
Best num_boost_round: 963
Best CV score: 1.117628092067576
CPU times: user 1h 15min 35s, sys: 17min 11s, total: 1h 32min 47s
Wall time: 5min 49s


In [251]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 23min 17s, sys: 5min 5s, total: 28min 23s
Wall time: 1min 46s


In [252]:
feature_importance(lgb_model).tail(30)

,feat_name,feat_imp
101,bmi_pit,1240
174,bmi_dif_p-c,1351
181,rank_x_year_dif_b-c,1412
103,slider,1426
182,bmi_dif_b-c,1504
140,batter_cnt,1506
0,pitch_cnt_in_game,1587
166,bmi_dif_p-b,1589
196,predict_right_ball,1672
42,elapsed_min,1712


### 予測

In [253]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 6min 27s, sys: 2.92 s, total: 6min 30s
Wall time: 26 s


## 結果出力

In [254]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.451574,0.081872,0.210515,0.128828,0.036124,0.000294,0.000046,0.090748
1,1,0.601114,0.082245,0.147742,0.104520,0.004938,0.000104,0.000017,0.059321
2,2,0.415409,0.102493,0.208843,0.102154,0.020048,0.000380,0.000064,0.150608
3,3,0.402481,0.155356,0.194838,0.152042,0.017500,0.000245,0.000032,0.077507
4,4,0.229547,0.062215,0.414926,0.118246,0.013473,0.000243,0.000045,0.161303


### CSV出力

In [255]:
#submit.to_csv(SUBMIT, header=False, index=False)

### Feather出力

In [256]:
submit_f = submit.drop(columns=['index'])
submit_f.rename(columns={
        0: 'predict_straight', 1: 'predict_curve', 2: 'predict_slider', 3: 'predict_shoot',
        4: 'predict_fork', 5: 'predict_changeup', 6: 'predict_sinker', 7: 'predict_cutball'
    }, inplace=True)
submit_f.to_feather(SUBMIT_F)
print(SUBMIT_F)

submit/22A/ball_22A_4.f


### 結果まとめ

In [257]:
ave_result = (sample_No == 4)

In [266]:
if ave_result:
    result = 'submit/{}/ball_{}_{}.f'.format(model_No, model_No, 1)
    print(result)
    df = pd.read_feather(result)
    columns = ['predict_straight', 'predict_curve', 'predict_slider', 'predict_shoot', 
               'predict_fork', 'predict_changeup', 'predict_sinker', 'predict_cutball']

    for i in range(2,5):
        result = 'submit/{}/ball_{}_{}.f'.format(model_No, model_No, i)
        print(result)
        temp = pd.read_feather(result)
        for c in columns:
            df[c] = df[c] + temp[c]

    for c in columns:
        df[c] = df[c]/4

    # 出力
    df = df.reset_index()
    df.to_csv(SUBMIT, header=False, index=False)
    print(SUBMIT)

submit/22A/ball_22A_1.f
submit/22A/ball_22A_2.f
submit/22A/ball_22A_3.f
submit/22A/ball_22A_4.f
submit/22A/ball_22A_sub.csv


### コース予測で使用

In [259]:
%%time
if not use_sub_model:
    train_predict = lgb_model.predict(train_d, num_iteration = lgb_model.best_iteration)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs


In [260]:
if not use_sub_model:
    df_train_predict = pd.DataFrame(train_predict).reset_index()
    submodel = pd.concat([df_train_predict, submit], ignore_index=True)
    submodel.drop(columns=['index'], inplace=True)
    submodel.rename(columns={
        0: 'predict_straight', 1: 'predict_curve', 2: 'predict_slider', 3: 'predict_shoot', 
        4: 'predict_fork', 5: 'predict_changeup', 6: 'predict_sinker', 7: 'predict_cutball'
    }, inplace=True)
    print(submodel.shape)
    submodel.head()

In [261]:
if not use_sub_model:
    submodel.to_feather(OUT_SUBMODEL)

In [262]:
column_cnt = len(train_d.columns)

### SIGNATE

In [267]:
'signate submit --competition-id=275 ./{} --note feat={}'.format(SUBMIT, column_cnt)

'signate submit --competition-id=275 ./submit/22A/ball_22A_sub.csv --note feat=197'

### GCS

In [268]:
'gsutil cp ./{} gs://yasunobu_baseball/submit/'.format(SUBMIT)

'gsutil cp ./submit/22A/ball_22A_sub.csv gs://yasunobu_baseball/submit/'

### 評価結果
#### 22A_sub (197) ... 22相当に戻す
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- (1) Best CV score: 1.1213900804295551
- (2) Best CV score: 1.123470119554106
- (3) Best CV score: 1.1194275868627095
- (4) Best CV score: 1.117628092067576
- 評価結果  : 1.3587044

#### 22A (187) ... 22相当に戻す
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- (1) Best CV score: 1.115960506369753
- (2) Best CV score: 1.1196612277859819
- (3) Best CV score: 1.1173714755877482
- (4) Best CV score: 1.1190928489075598
- 評価結果  : 1.3548899

#### 22_sub_tune (197) 
- 'num_leaves' : 17,
- 'learning_rate' : 0.1,
- (1) Best CV score: 1.113600607662189
- (2) Best CV score: 1.1167149593443422
- (3) Best CV score: 1.1149536663781887
- (4) Best CV score: 1.1169140944754492
- 評価結果  : 1.3542559

#### 23_sub (197) 
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- (1) Best CV score: 1.1218386395343705
- (2) Best CV score: 1.1240501409693007
- (3) Best CV score: 1.1197733797599718
- (4) Best CV score: 1.1170595045765104
- 評価結果 : 1.3716876

#### 23 (187) ... 2017年の集計がない選手はNaNに
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- (1) Best CV score: 1.1168878665056097
- (2) Best CV score: 1.1201824919009344
- (3) Best CV score: 1.1176435965681746
- (4) Best CV score: 1.119491352126086
- 評価結果 : 未提出

#### 22_sub2 (197) ... 特徴量で集計した部分を抜く
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- (1) Best CV score: 1.1135086240261998
- (2) Best CV score: 1.1165352243946294
- (3) Best CV score: 1.1148759371482442
- (4) Best CV score: 1.1168424908397538
- 評価結果 : 1.3529450 ★

#### 22_sub (197) 
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- (1) Best CV score: 1.1075510468720624
- (2) Best CV score: 1.106096699487912
- (3) Best CV score: 1.1066375157863488
- (4) Best CV score: 1.106245387987554
- 評価結果  : 1.3530160

#### 22 (187) ... 1/4ずつ予測して平均
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- (1) Best CV score: 1.109870194259073
- (2) Best CV score: 1.1089544307555284
- (3) Best CV score: 1.109279132521207
- (4) Best CV score: 1.1078911484363843
- 評価結果  : 1.3574360

#### 21_select (147) ... FI下位50個を削除
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1172
- Best CV score: 1.0989019423650523
- Wall time: 9min 30s
- 評価結果  : 1.4009009

#### 21_sub (208) ... 球種のサブモデルの結果をそのまま使った
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1021
- Best CV score: 0.812901550823909
- Wall time: 8min 17s
- 評価結果  : 1.6952410

#### 20_sub (176) 
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1328
- Best CV score: 1.097496116910761
- Wall time: 14min 8s -> 8min 26s(n1-standard-16)
- 評価結果  : 1.4025144

#### 20 (166) ... player同士の組み合わせを削除
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1263
- Best CV score: 1.098749368845937
- Wall time: 12min 30s
- 評価結果  : 1.3964075

#### 19_sub (200) 
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1199
- Best CV score: 1.098298504761071
- Wall time: 14min 16s -> 7min 23s(n1-standard-16)
- 評価結果  : 1.3945102

#### 19 (190) ... コースの種類のみ
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1141
- Best CV score: 1.1004105063817775
- Wall time: 12min 53s
- 評価結果  : 1.3974324

#### 18_sub (230) 
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1204
- Best CV score: 1.1004215512272495
- Wall time: 15min 34s
- 評価結果  : 1.3953962

#### 18 (217) ... コースの種類、1球前のボール、ファウル数
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1102
- Best CV score: 1.1004234715781385
- Wall time: 13min 42s
- 評価結果  : 1.3954167

#### 17 (191) ... イニング時間、打席時間、平均投球間隔(差)、サヨナラ、延長戦
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1100
- Best CV score: 1.1034107856339535
- Wall time: 26min 44s
- 評価結果  : 1.3963057

#### 16 (181) ... 選手IDごとの2017年のコース実績
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1178
- Best CV score: 1.1021930539837317
- Wall time: 12min 35s
- 評価結果  : 1.3993836

#### 15 (153) ... 左右で球種/コースを別に集計
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1189
- Best CV score: 1.1013869435441306
- Wall time: 11min 9s
- 評価結果  : 未提出

#### 14 (153) ... 左右で球種を別に集計
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1240
- Best CV score: 1.1021952294615331
- Wall time: 11min 34s
- 評価結果  : 1.3981900

#### 13 (157) ... 前の投球からの時間差を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1416
- Best CV score: 1.1062463941695753
- Wall time: 13min 10s
- 評価結果  : 1.4012184

#### 12 (156) ... コースの集計を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1353
- Best CV score: 1.106942086205103
- Wall time: 27min 41s
- 評価結果  : 1.4016077

#### 11 (143) ... ボールカウントの集計を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1209
- Best CV score: 1.1075153721728936
- Wall time: 23min 19s
- 評価結果  : 未提出

#### 10_tune (135) ... 再チューニング
- 'num_leaves' : 15,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1260
- Best CV score: 1.1118897437428692
- Wall time: 8min 40s
- 評価結果  : 1.4060397

#### 10s (135) ... trainを75%サンプリング
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1412
- Best CV score: 1.1190287120710383
- Wall time: 12min 5s
- 評価結果  : 1.4310869

#### 10 (135) ... 5相当に戻した
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1112
- Best CV score: 1.111265000190272
- Wall time: 9min 30s
- 評価結果  : 1.4008712

#### 9 (136) ... 5相当に戻した=>NG
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1106
- Best CV score: 1.1113177029272225
- Wall time: 25min 44s (local)
- 評価結果  : 1.5133521

#### 8 (157)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1100
- Best CV score: 1.1093728644236824
- Wall time: 10min 4s
- 評価結果  : 1.4925832

#### 7 (235)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1204
- Best CV score: 1.108590082437519
- Wall time: 12min 56s
- 評価結果  : 1.4946937

#### 6 (207)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1135
- Best CV score: 1.1086837935965734
- Wall time: 11min 54s
- 評価結果  : 1.3999194

#### 5-tune (参考)
- 'num_leaves' : 18,
- 'learning_rate' : 0.05,
- Best num_boost_round: 2395
- Best CV score: 1.1103399384119979
- Wall time: 18min 29s (n1-standard-8)

#### 5-tune (135)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- CV score: 1.1112402566616455
- 評価結果  : 1.3991780
- CV Wall time: 27min 48s (local) -> 18min 24s (n1-standard-4) -> 9min 3s (n1-standard-8)

#### 5 (135)
- 'num_leaves' : 31,
- 'learning_rate' : 0.1,
- CV score: 1.1161825304353852
- 評価結果  : 1.4252286

#### 4 (128)
- CV score: 1.1158111693845418
- 評価結果  : 1.4299265

#### 3 (97)
- CV score: 1.1172290313076036
- 評価結果  : 1.4335806